In [1]:
import csv 
import numpy as np
import cv2
from scipy import ndimage

In [2]:
lines =[]
with open("data4/driving_log.csv") as f:
    reader=csv.reader(f)
    for line in reader:
        lines.append(line)

In [3]:
images = []
measurements =[]

for data in lines:
    filename = data[0]
    image=ndimage.imread(filename)
    images.append(image)
    measurement = float(data[3])
    measurements.append(measurement)

/lhome/rkukrej/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  


In [4]:
print("Total images {0} and measurements {1}".format(len(images),len(measurements)))

Total images 471 and measurements 471


In [5]:
augmented_images,augmented_measurements=[],[]
for image,measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement)
    augmented_measurements.append(measurement*-1.0)
    
    

In [6]:
x_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)
#x_train = np.array(images)
#y_train = np.array(measurements)

In [2]:
from keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,Dropout,Lambda,AveragePooling2D
from keras.models import Sequential
from keras.losses import mse
from keras.optimizers import Adadelta,adam
from keras import initializers
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


ImportError: No module named tensorflow

In [11]:
input_shape = (160,320,3)
batch_size = 16
epochs = 5

In [12]:
#model = Sequential()
#model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
#model.add(Conv2D(32, kernel_size=(3, 3),
#                 activation='relu',kernel_initializer='random_uniform',
#                bias_initializer='zeros'))
#model.add(BatchNormalization())
#model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(Dense(1))
#More deep model
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())

model.add(Flatten())

model.add(Dense(units=120, activation='relu'))

model.add(Dense(units=84, activation='relu'))

model.add(Dense(units=1))

In [13]:
model1 = Sequential()
model1.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model1.add(Flatten())
model1.add(Dense(1,kernel_initializer='random_uniform', bias_initializer='zeros'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 158, 318, 6)       168       
_________________________________________________________________
average_pooling2d_1 (Average (None, 79, 159, 6)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 77, 157, 16)       880       
_________________________________________________________________
average_pooling2d_2 (Average (None, 38, 78, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 47424)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               5691000   
__________

In [15]:
model.compile(loss="mse",
              optimizer="adam",
              metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2,shuffle=True)

Train on 753 samples, validate on 189 samples
Epoch 1/5
753/753 [==============================] - 16s 21ms/step - loss: 0.3568 - acc: 0.5060 - val_loss: 0.1320 - val_acc: 0.7778
Epoch 2/5
753/753 [==============================] - 16s 21ms/step - loss: 0.1495 - acc: 0.5564 - val_loss: 0.1586 - val_acc: 0.7778
Epoch 3/5
753/753 [==============================] - 16s 21ms/step - loss: 0.1098 - acc: 0.6122 - val_loss: 0.1966 - val_acc: 0.7725
Epoch 4/5
753/753 [==============================] - 16s 21ms/step - loss: 0.0746 - acc: 0.6401 - val_loss: 0.2879 - val_acc: 0.5291
Epoch 5/5
753/753 [==============================] - 19s 25ms/step - loss: 0.0640 - acc: 0.6494 - val_loss: 0.2539 - val_acc: 0.7725


In [17]:
model.save('model.h5')
